In [1]:
from google.colab import drive
drive.mount('/content/drive')
print("Google Drive montado.")

Mounted at /content/drive
Google Drive montado.


In [12]:
# --- 0. Importações de Bibliotecas Essenciais ---
import pandas as pd
import numpy as np
import torch
from transformers import AutoTokenizer, AutoModel, AutoConfig # Importar classes para DNABERT
import os
import sys
from tqdm import tqdm # Para barras de progresso

In [18]:
# --- 1. Configurações de Ficheiros e Modelo ---

BASE_PATH = "/content/drive/MyDrive"
TRAINING_DATA_FILE = os.path.join(BASE_PATH, "training_data.csv")
DNABERT_EMBEDDINGS_OUTPUT_FILE = os.path.join(BASE_PATH, "dnabert_embeddings.npy")

DNABERT_MODEL_NAME = 'zhihan1996/DNABERT-S'

BATCH_SIZE = 16
MAX_SEQUENCE_LENGTH = 512

In [19]:
# --- 2. Carregar Dados ---
print(f"--- A carregar dados do ficheiro: {TRAINING_DATA_FILE} ---")
try:
    df = pd.read_csv(TRAINING_DATA_FILE)
    if 'sequence' not in df.columns:
        print("ERRO: O ficheiro CSV não contém uma coluna 'sequence'. Por favor, verifique o nome da coluna.")
        sys.exit(1)
    sequences = df['sequence'].tolist()
    print(f"Total de {len(sequences)} sequências carregadas.")
except FileNotFoundError:
    print(f"ERRO: O ficheiro '{TRAINING_DATA_FILE}' não foi encontrado.")
    print("Por favor, verifique o caminho e certifique-se de que o ficheiro está acessível.")
    sys.exit(1)
except Exception as e:
    print(f"ERRO ao carregar o ficheiro CSV: {e}")
    sys.exit(1)

--- A carregar dados do ficheiro: /content/drive/MyDrive/training_data.csv ---
Total de 9336 sequências carregadas.


In [20]:
# --- 3. Carregar Tokenizer e Modelo DNABERT ---
print(f"--- A carregar tokenizer e modelo DNABERT: {DNABERT_MODEL_NAME} ---")
try:
    # Carregar tokenizer com trust_remote_code
    tokenizer = AutoTokenizer.from_pretrained(DNABERT_MODEL_NAME, trust_remote_code=True)

    # Carregar configuração e modelo com trust_remote_code e low_cpu_mem_usage
    config = AutoConfig.from_pretrained(DNABERT_MODEL_NAME, trust_remote_code=True, low_cpu_mem_usage=True)

    model = AutoModel.from_pretrained(DNABERT_MODEL_NAME, trust_remote_code=True, config=config)

    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
    model.to(device)
    model.eval() # Colocar o modelo em modo de avaliação (desativa dropout, etc.)
    print(f"Modelo DNABERT carregado e movido para: {device}")
except Exception as e:
    print(f"ERRO ao carregar o modelo/tokenizer DNABERT: {e}")
    print("Por favor, certifique-se de que tem ligação à internet para descarregar o modelo, ou que o modelo está em cache local.")
    print("Se o erro persistir, pode ser necessário instalar 'flash_attn' se use_flash_attention=True estiver implícito.")
    sys.exit(1)

--- A carregar tokenizer e modelo DNABERT: zhihan1996/DNABERT-S ---


/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/268 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/168k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/125 [00:00<?, ?B/s]

...

flash_attn_triton.py:   0%|          | 0.00/42.7k [00:00<?, ?B/s]

A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-S:
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
A new version of the following files was downloaded from https://huggingface.co/zhihan1996/DNABERT-S:
- bert_layers.py
- bert_padding.py
- flash_attn_triton.py
. Make sure to double-check they do not contain any added malicious code. To avoid downloading new versions of the code file, you can pin a revision.
/usr/local/lib/python3.11/dist-packages/huggingface_hub/file_download.py:943: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


pytorch_model.bin:   0%|          | 0.00/468M [00:00<?, ?B/s]

/root/.cache/huggingface/modules/transformers_modules/zhihan1996/DNABERT-S/2efd650282ec5d5ab377c787c76ea56b723f6b7c/bert_layers.py:126: UserWarning: Unable to import Triton; defaulting MosaicBERT attention implementation to pytorch (this will reduce throughput when using this model).
  warnings.warn(


Modelo DNABERT carregado e movido para: cpu


In [24]:
# --- 4. Função para Gerar Embeddings em Lotes ---
def get_embeddings_batch(sequences_batch, tokenizer, model, device):
    """
    Gera embeddings para um lote de sequências, replicando a lógica da classe DNABERTEmbedder.
    """
    tokens = tokenizer(sequences_batch, return_tensors="pt", truncation=True, max_length=MAX_SEQUENCE_LENGTH, padding=True)

    for key, value in tokens.items():
        if value.dtype == torch.long:
            tokens[key] = value.to(device)
        else:
            tokens[key] = value.to(device).to(torch.float16) # Conforme a sua classe

    with torch.no_grad():
        output = model(**tokens)[0]


    # Faz a média dos embeddings das tokens ao longo da dimensão da sequência
    mean_embeddings = output.mean(dim=1)

    if mean_embeddings.dim() == 1:
        mean_embeddings = mean_embeddings.unsqueeze(0) # Adiciona a dimensão do batch de volta se for 1D

    torch.cuda.empty_cache() # Liberar memória da GPU (conforme sua classe)
    return mean_embeddings.cpu().numpy()

In [25]:
# --- 5. Gerar e Guardar Todos os Embeddings ---
print("--- A gerar embeddings para todas as sequências ---")
all_embeddings = []

for i in tqdm(range(0, len(sequences), BATCH_SIZE), desc="Gerando Embeddings"):
    batch_sequences = sequences[i:i + BATCH_SIZE]
    embeddings_batch = get_embeddings_batch(batch_sequences, tokenizer, model, device)
    all_embeddings.append(embeddings_batch)

final_embeddings = np.vstack(all_embeddings)

--- A gerar embeddings para todas as sequências ---


Gerando Embeddings: 100%|██████████| 584/584 [20:41<00:00,  2.13s/it]


In [26]:
# --- 6. Guardar Embeddings no Ficheiro .npy ---
try:
    np.save(DNABERT_EMBEDDINGS_OUTPUT_FILE, final_embeddings)
    print(f"\nEmbeddings guardados com sucesso em: {DNABERT_EMBEDDINGS_OUTPUT_FILE}")
    print(f"Dimensão dos embeddings finais: {final_embeddings.shape}")
except Exception as e:
    print(f"ERRO ao guardar os embeddings: {e}")
    sys.exit(1)

print("\nScript de geração de embeddings concluído.")


Embeddings guardados com sucesso em: /content/drive/MyDrive/dnabert_embeddings.npy
Dimensão dos embeddings finais: (9336, 768)

Script de geração de embeddings concluído.
